In [1]:
import os
import re
from fnmatch import fnmatch
from collections import defaultdict

# Specify the directory to search
search_directory = "C:\\Users\\inbra\\OneDrive - Software AG\\Documents\\Adda\\go-live\\09-04-2023"

# Define a dictionary to store counts
url_counts = defaultdict(lambda: {"200": 0, "non-200": []})

# Function to process http.log files
def process_http_log_file(file_path):
    with open(file_path, 'r') as f:
        log_lines = f.readlines()

    for line in log_lines:
        # Find the position of the first date string
        date_match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', line)
        if date_match:
            date_start = date_match.start()
            remaining_part = line[date_start:]
            
            # Split the remaining part into columns
            columns = remaining_part.split()

            if len(columns) >= 7:
                url = columns[3]  # URL is at index 3
                response_code = columns[4]  # Response code is at index 4
                
                # Ignore the last part of the URL if it contains numbers
                url_parts = url.split('/')
                if url_parts[-1].isdigit():
                    url = '/'.join(url_parts[:-1])
                
                if response_code == "200":
                    url_counts[url]["200"] += 1
                else:
                    message = line.strip()  # Entire line as the message
                    url_counts[url]["non-200"].append(message)

# Recursively search for http.log files in the directory and its subdirectories
for root, dirs, files in os.walk(search_directory):
    for file in files:
        if fnmatch(file, "http*.log"):
            file_path = os.path.join(root, file)
            process_http_log_file(file_path)

# Print the results
for url, counts in url_counts.items():
    count_200 = counts["200"]
    count_non_200 = len(counts["non-200"])  # Count of non-200 responses
    non_200_messages = counts["non-200"]
    
    print(f"URL: {url}")
    print(f"Count of 200 responses: {count_200}")
    print(f"Count of Non-200 responses: {count_non_200}")
    print(f"Non-200 responses:")
    for message in non_200_messages:
        print(message)
    print("-" * 30)


URL: /
Count of 200 responses: 22274
Count of Non-200 responses: 0
Non-200 responses:
------------------------------
URL: /TammJourneyMaqta/aribaDiscovery/events
Count of 200 responses: 0
Count of Non-200 responses: 259
Non-200 responses:
"10.242.97.57, 10.242.97.130" 2023-08-07 10:01:31 GET /TammJourneyMaqta/aribaDiscovery/events 404 https://adda-msr-svc-cip:5543 192.168.32.65 APIGW:41c0137c-e315-4079-a804-8697d9c01d06:51619 66 113
"10.242.97.57, 10.242.97.45" 2023-08-07 10:01:31 GET /TammJourneyMaqta/aribaDiscovery/events 404 https://adda-msr-svc-cip:5543 192.168.32.49 APIGW:8b2d9135-8d22-47ef-bb59-47baea53d944:51910 48 113
"10.242.97.57, 10.242.97.238" 2023-08-07 10:01:31 GET /TammJourneyMaqta/aribaDiscovery/events 404 https://adda-msr-svc-cip:5543 192.168.32.97 APIGW:4545e9e2-21b0-4c92-8310-91dabb6e7aff:62836 49 113
"10.242.97.57, 10.242.97.130" 2023-08-07 10:01:31 GET /TammJourneyMaqta/aribaDiscovery/events 404 https://adda-msr-svc-cip:5543 192.168.32.97 APIGW:31006147-853d-47b2-8